# Lesson 06. Introduction to Patterns in Multivariate Data

## Introduction
In this exercise, we will explore how Principal Component Analysis (PCA) — also known in climate science as Empirical Orthogonal Function (EOF) analysis — can be used to uncover dominant patterns of variability in sea surface temperature (SST).

Using the Extended Reconstructed Sea Surface Temperature (ERSSTv6) dataset, we will:

1. Compute SST anomalies relative to a climatological baseline.
2. Construct the Niño 3.4 Index, a widely used measure of the El Niño–Southern Oscillation (ENSO).
3. Apply PCA to the tropical Pacific SST field and interpret the resulting modes (EOFs and PCs).

By the end of this lesson, we should be able to:
- Describe how PCA decomposes a large spatiotemporal dataset into spatial patterns and temporal modes.
- Recognize the seasonal cycle and ENSO as the first two leading EOFs of tropical Pacific SST.
- Compare a physically defined ENSO index (Niño 3.4) with a statistically derived one (PC 2).


#### Acknowledgements and AI Disclaimer 
This lesson was adapted from the [Karen L. Smith](https://kls2177.github.io/)'s book on [Climate and Geophysical Data Analysis](https://kls2177.github.io/Climate-and-Geophysical-Data-Analysis/chapters/index.html), in particular, the chapter on [Patterns in Multivariate Data](https://kls2177.github.io/Climate-and-Geophysical-Data-Analysis/chapters/Week7/pca.html). Data for this lecture is downloaded and complied from [NOAA's Extended Reconstructed Sea Surface Temperature (ERSST)](https://www.ncei.noaa.gov/products/extended-reconstructed-sst). 

A set of [downloader](preprocessor_download-sst.sh) and [combiner scripts](preprocessor_combine-nc.ipynb) are provided in this repository. For your convenience, I have compiled the data (1950-2024) [in this link](https://drive.google.com/file/d/1GIY4GuzoVK7qTlhfMywEMS7ghmywiJ_7/view?usp=sharing). Download this and store the data in the same folder as this notebook.

The material has been updated and restructured for Meteo 203 (Methods of Analytical Meteorology & Oceanography) with the assistance of ChatGPT for code modernization, annotation, and formatting.

All scientific content, computations, and instructional revisions have been reviewed and verified by BBR.

In [ ]:
# in case packages haven't been installed, uncomment the snippet below, and run the cell.
# !conda install -c conda-forge h5py xarray netCDF4 --name meteo203 -y

In [ ]:
# Imports
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from cartopy.geodesic import Geodesic
from shapely.geometry import box


---

### Part 1. Open the data using xarray

In [ ]:
file_path = 'ersst.v6.195001_202412.nc'
ds = xr.open_dataset(file_path)

#### Inspect the data

In [ ]:
ds

We'll be selecting `sst` data for the first level using the snippet below.

In [ ]:
sst = ds['sst'].isel(lev=0)

In [ ]:
sst

---

### Part 2. Calculate climatologies.

The data we have is for 1950 to 2024. Let's calculate for climatological normals from 1950 to 1979 by calculating the mean according to the `time` dimension. 

We first get the time period using `.sel(time=slice("1950", "1979"))`, and then the mean in the time axis using `.mean(dim="time")`. This can be done in one line as provided in the script below.

In [ ]:
sst_clim = sst.sel(time=slice("1950", "1979")).mean(dim="time")

We can now plot the climatologies.

In [ ]:
plt.figure(figsize=(12, 5))
sst_clim.plot(cmap='RdBu_r', vmin=0, vmax=30)
plt.title("ERSSTv6 Sea Surface Temperature Climatology (1950-1979)")
plt.show()


---
### Part 3. Calculate annual means.

The data is stored in monthly periods (`time: 900`). We can index and select according to month, or we can do this according to year. For this section let's slice according to year (`.sel(time="2024")`) giving us 12 months in a dataframe.  We can then proceed to calculate the annual mean, again according to `time`.

In [ ]:
# Get the SST for one year, and then calculate the mean.
sst_2024 = sst.sel(time="2024").mean(dim='time')

In [ ]:
plt.figure(figsize=(12, 5))
sst_2024.plot(cmap='RdBu_r', vmin=0, vmax=30)
plt.title("ERSSTv6 Annual Mean Sea Surface Temperature (2024)")
plt.show()


---
### Part 4. Calculate anomalies.

By itself the annual means do not look much. By qualitative comparison, the two plots above look similar. To see the differences of a certain year compared to the climatology, we can calculate the *anomalies* by subtracting the climatology (1950-1979) from the annual mean. Notice how the colorbar is now centered on zero, depending on how we set the `vmin` and `vmax`.

In the map below, notice where the SST is warmer than the climatological normals. 

In [ ]:
sst_anom_2024 = sst_2024 - sst_clim

In [ ]:
plt.figure(figsize=(12,5))
sst_anom_2024.plot(cmap='RdBu_r', vmin=-2, vmax=2)
plt.title("SST Anomalies (2024 relative to 1950-1979 climatology)")
plt.show()

---
### Part 5. Calculate anomalies for the 1997 ENSO onset.

The 1997-1998 ENSO was considered [one of the strongest on record](https://www.pmel.noaa.gov/pubs/outstand/mcph2029/text.shtml). Let's try to take a look at the annual SST anomalies for 1997.

First let's calculate and plot the annual average SSTs for 1997.

In [ ]:
sst_1997 = sst.sel(time="1997").mean(dim='time')

plt.figure(figsize=(12, 5))
sst_1997.plot(cmap='RdBu_r', vmin=0, vmax=30)
plt.title("ERSSTv6 Annual Mean Sea Surface Temperature (1997)")
plt.show()


Now let's calculate for the anomalies and then plot. 

How does the map above (annual mean) compare with the map below (anomalies)? Notice the extension of the warm tongue in South America. 

In [ ]:
# Get the SST for one year
sst_anom_1997 = sst_1997 - sst_clim
plt.figure(figsize=(12,5))
sst_anom_1997.plot(cmap='RdBu_r', vmin=-2, vmax=2)
plt.title("SST Anomalies (1997 relative to 1950-1979 climatology)")
plt.show()

---
### Part 6. Plot in cartopy to contextualize

The maps above are simple array plots. We can plot them with cartopy so we have contextual information (i.e. country boundaries). 

Hint: try changing the `central_longitude` input, what happens?

In [ ]:

# Select and compute anomaly
sst_1997 = sst.sel(time="1997").mean(dim="time")
sst_anom_1997 = sst_1997 - sst_clim

# Set up figure with Cartopy
fig = plt.figure(figsize=(12,5))
proj = ccrs.PlateCarree(central_longitude=0)
ax = plt.axes(projection=proj)

# Add map features
ax.coastlines(resolution="110m", linewidth=1)
ax.add_feature(cfeature.BORDERS, linestyle=":")
ax.set_global()
ax.gridlines(draw_labels=True, linewidth=0.3, color="gray", alpha=0.5)

# Plot SST anomalies
pcm = sst_anom_1997.plot.pcolormesh(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="RdBu_r",
    vmin=-2, vmax=2,
    add_colorbar=True,
    add_labels=False
)

# Add title
plt.title("SST Anomalies (1997 vs 1950-1979 climatology)", fontsize=12)
plt.show()


---
### Part 7. Calculating the Niño3.4 Index.

In this sectopm we are going to construct the ENSO index. First, we will construct it following the [Niño 3.4 Index](http://www.cgd.ucar.edu/cas/catalog/climind/TNI_N34/index.html#Sec5) definition. The Niño3.4 Index is a commonly used metric of ENSO variability. The recipe for calculating it is:

1. Compute area averaged total SST from Niño3.4 region (5N-5S, 170W-120W).
2. Compute monthly climatology (1950-1979) for area averaged total SST from Niño 3.4 region
3. Subtract climatology from area averaged total SST time series to obtain anomalies.
4. Smooth the anomalies with a 5-month running mean.
5. Standardize the smoothed Niño3.4 by its standard deviation over the climatological period 1950-1979.

But first, let's plot the `sst_1997` maps from above along with the  Niño 3.4 boundaries. We can plot the [Niño 3.4 boundaries](https://www.ncei.noaa.gov/access/monitoring/enso/sst) using the `n34_box` variable and `add_geometries` in the script below. Since we have already calculated `sst_anom_1997` in a cell above, no need to recalculate here. 

In [ ]:
# Set up figure with Cartopy
fig = plt.figure(figsize=(12,5))
proj = ccrs.PlateCarree(central_longitude=180)
ax = plt.axes(projection=proj)

# Add map features
ax.coastlines(resolution="110m", linewidth=1)
ax.add_feature(cfeature.BORDERS, linestyle=":")
ax.set_global()
ax.gridlines(draw_labels=True, linewidth=0.3, color="gray", alpha=0.5)

# Plot SST anomalies
pcm = sst_anom_1997.plot.pcolormesh(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="RdBu_r",
    vmin=-2, vmax=2,
    add_colorbar=True,
    add_labels=False
)

# Define Nino 3.4 box boundaries
lon_min, lon_max = 190, 240   # 170°W–120°W in 0–360° convention
lat_min, lat_max = -5, 5

# Create a rectangular polygon
n34_box = box(lon_min, lat_min, lon_max, lat_max)

# Add it to the map
ax.add_geometries(
    [n34_box],
    crs=ccrs.PlateCarree(),
    facecolor="none",
    edgecolor="black",
    linewidth=2,
    linestyle="--"
)


# Add title
plt.title("SST Anomalies (1997 vs 1950-1979 climatology) with Niño 3.4 Boundaries", fontsize=12)
plt.show()


At this point let's now calculate the SST anomalies in the Niño 3.4 region. We can use xarray's [`sel`](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.sel.html) and then slice according to `lat` and `lon`.  Afterwards, we can take the mean for the sliced box using `.mean(dim=["lat, "lon"])`.

Notice in the script below we directly subtracted the slice from the climatology. If the xarray dataframe is constructed properly, the mathematical operations will respect the geography of the individual dataframes (i.e. we are subtracting from the same slices). 

In [ ]:
# sst_1997_monthly = sst.sel(time="1997")
sst_anom_n34 = sst.sel(lat=slice(-5, 5), lon=slice(190, 240)) - sst_clim

# Take the mean over both lat and lon for each time step
nino34 = sst_anom_n34.mean(dim=["lat", "lon"])


At this point, `nino34` should be a time-series array of 900 timesteps. Question: what do these timesteps represent?

In [ ]:
# print the nino34 dataframe
nino34

[The Niño 3.4 index typically uses a 5-month running mean](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni), and El Niño or La  Niña events are defined when the  Niño 3.4 SSTs exceed +/- 0.4C for a period of six months or more. Let's now calculate for the 5-month running mean using xarray's [`rolling`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.rolling.html) windows, and then let's calculate the `.mean()`.

In [ ]:
# Apply 5-month centered rolling mean
nino34_smooth = nino34.rolling(time=5, center=True).mean()


Since the data spans many years (1950-1979), we need to first normalize and then  standardize; in our lecture, we called this the *standardized anomalies*. For the `base_period` 1950 to 1979, let's first calculate the mean for the period (`mean_base`), and then the standard deviation (`std_base`). Afterwards, let's calculate the standardized anomalies by subtracting the mean from the smoothed data, and then dividing by the standard deviation for the base period.

In [ ]:
# Compute mean and std from baseline (1950–1979)
base_period = slice("1950", "1979")
mean_base = nino34_smooth.sel(time=base_period).mean()
std_base  = nino34_smooth.sel(time=base_period).std()

# Standardize
nino34_std = (nino34_smooth - mean_base) / std_base


At this point we can plot the `nino34_std`, which is the standardized anomalies for the 5-month running mean. [The Oceanic Niño Index or ONI](https://www.ncei.noaa.gov/access/monitoring/enso/sst#oni) defines warm and cold phases as a minimum of five consecutive 3-month running averages of SST anomalies in the Niño 3.4 region surpassing a threshold of +/- 0.5°C, respectively. 

We can plot these thresholds using matplotlib's [`axhline`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.axhline.html). We can also color the under the time-series whenever they exceed 0 in the positive or negative using [`fill_between`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.fill_between.html). All of this code is provided below.

In [ ]:
plt.figure(figsize=(12,4))
nino34_std.plot(color="steelblue", lw=1.5, label="Niño 3.4 (5-month mean)")
plt.axhline(0.5, color="r", ls="--", lw=1)
plt.axhline(-0.5, color="b", ls="--", lw=1)
plt.fill_between(nino34_std["time"], 0, nino34_std,
                 where=nino34_std > 0, color="red", alpha=0.3)
plt.fill_between(nino34_std["time"], 0, nino34_std,
                 where=nino34_std < 0, color="blue", alpha=0.3)
plt.title("Standardized Niño 3.4 Index (5-month running mean, base 1950–1979)")
plt.ylabel("Standardized SST Anomaly (σ)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()


Congratulations! You have plotted the Nino 3.4 index. 

Based on the plot above, is the 1997-1998 still the strongest El Niño event? If you want, you can try calculating and plotting other El Niño events in additional cells below. 

---
### Part 8. PCA of Tropical Pacific SST: Setting up the Problem

Now we will perform a principal component analysis (PCA) on the tropical Pacific SST. Our goal is to see if we can recreate the Niño 3.4 Index time series above through PCA. 

Before we do the PCA, we will take a look at the spatial structure of the data. Let’s pick a larger region so that we can more easily see what is going on.

In [ ]:
sst_pac = sst.sel(time=slice("1950", "2020"),
                  lat=slice(-30, 30),
                  lon=slice(120, 300))

The output of PCA is a set of pairs of spatial patterns (EOFs) and time series (PCs). The size of the set depends on the size of the original data. The pairs are ordered in terms of variance explained, i.e. the first pair explains the largest fraction of variance in the data, the second pair explains the second largest fraction of variance, and so on.

If we leave the seasonal cycle in our data, the seasonal cycle will likely emerge as the largest source of variance. Let’s see if this is what we get.

First, we will compute an anomaly. We will NOT subtract a monthly climatology (we want to see the seasonal cycle emerge from our analysis) - instead just subtract the time mean.

In [ ]:
# (a) Remove only the time mean
ssta_pac = sst_pac - sst_pac.mean(dim="time")

To visualize these anomalies, we can plot the data on a map by slicing just one one element in time.

In [ ]:
plt.figure(figsize=(10,5))
plt.pcolormesh(ssta_pac[0],cmap = "RdBu_r", )
plt.clim(-4,4)
plt.colorbar()
plt.show()

If we do remove the monthly climatology, how does the above anomaly plot change?

In [ ]:
# (1) Remove overall time mean — keep seasonal cycle
ssta_pac = sst_pac - sst_pac.mean(dim="time")

# (2) Remove monthly climatology — remove seasonal cycle
sst_clim_monthly = sst_pac.groupby("time.month").mean(dim="time")
ssta_pac_noseason = sst_pac.groupby("time.month") - sst_clim_monthly

plt.figure(figsize=(18,5))

plt.subplot(1,2,1)
ssta_pac.isel(time=0).plot(cmap="RdBu_r", vmin=-4, vmax=4)
plt.title("(a) SST Anomaly with Time Mean Removed")

plt.subplot(1,2,2)
ssta_pac_noseason.isel(time=0).plot(cmap="RdBu_r", vmin=-4, vmax=4)
plt.title("(b) SST Anomaly with Monthly Climatology Removed")

plt.tight_layout()
plt.show()


We see some similar features between the two plots, but we also see that the warm southern hemisphere and the cold northern hemisphere is reduced, suggesting that this part of the pattern is associated with the seasonal cycle.

When we subtract the time mean, we remove the global offset but the repeating annual cycle remains.

When we subtract the monthly climatology, we remove the seasonal cycle itself — leaving behind year-to-year anomalies like El Niño and La Niña.
This step is crucial because PCA identifies dominant patterns of variance, and if we don’t remove the seasonal cycle, it will dominate the first EOF.

PCA should identify sources of variance, so a good place to start to explore the variance in our data is to plot the standard deviation for each grid point.

We can plot the standard deviation on a map. 

When we only remove the overall mean, the largest SST variability appears in the mid-latitudes — that’s mostly the seasonal cycle.

But when we remove the monthly climatology, the remaining variability shifts to the tropical Pacific, where ENSO dominates.

In [ ]:
# Compute spatial standard deviation maps
ssta_pac_std = ssta_pac.std(dim="time")
ssta_pac_std_noseason = ssta_pac_noseason.std(dim="time")

# Visualize side by side
plt.figure(figsize=(18,5))

# (a) Time-mean removed (seasonal cycle retained)
plt.subplot(1,2,1)
ssta_pac_std.plot(cmap="Reds", vmin=0, vmax=5, add_colorbar=True)
plt.title("(a) STDEV of SST Anomaly (Time Mean Removed)")

# (b) Monthly climatology removed (seasonal cycle removed)
plt.subplot(1,2,2)
ssta_pac_std_noseason.plot(cmap="Reds", vmin=0, vmax=1.5, add_colorbar=True)
plt.title("(b) STDEV of SST Anomaly (Monthly Climatology Removed)")

plt.tight_layout()
plt.show()

We can then also plot this in a cartopy map


In [ ]:
proj = ccrs.PlateCarree(central_longitude=180)

fig, axs = plt.subplots(1, 2, figsize=(18,5), subplot_kw={'projection': proj})

for ax, field, title, vmax in zip(
    axs,
    [ssta_pac_std, ssta_pac_std_noseason],
    ["(a) Time Mean Removed", "(b) Monthly Climatology Removed"],
    [5, 1.5]
):
    field.plot.pcolormesh(
        ax=ax, transform=ccrs.PlateCarree(),
        cmap="Reds", vmin=0, vmax=vmax,
        add_colorbar=True, add_labels=False
    )
    ax.coastlines()
    ax.set_global()
    ax.set_title(f"STDEV of SST Anomaly {title}")

plt.tight_layout()
plt.show()


### Part 9. PCA: Step-by-step

In this section we will calculate the PCA using the provided steps below. For a very brief overview, you can take a look at a brief summary of calculating the PCA [here](https://medium.com/analytics-vidhya/understanding-principle-component-analysis-pca-step-by-step-e7a4bb4031d9). Take note that the aforementioned article calculates PCA for 2 dimensions, while we will be calculating for 3 dimensions (which we will be reducing to 2). 

In summary, the steps are as follows:
1. Clean the data by filling in missing values
2. Standardize the data
3. Convert from 3D to 2D
4. Calculate the [covariance matrix](https://www.itl.nist.gov/div898/handbook/pmc/section5/pmc541.htm)
5. Perform [eigenanalysis](https://online.stat.psu.edu/stat505/lesson/4/4.5) for the covariance matrix
6. Extract and standardize the first two [Empirical Orthogonal Functions or EOF](https://climatedataguide.ucar.edu/climate-tools/empirical-orthogonal-function-eof-analysis-and-rotated-eof-analysis)
7. Plot EOF1 (seasonal cycle), plot EOF2 (what other pattern emerges).

#### Step 1: Fill in missing values
This step is important since some matrix calculations will fail if some values are missing.

In [ ]:
a = ssta_pac.fillna(0)

#### Step 2: Standardize Data

In [ ]:
a = (a - a.mean()) / a.std()
print(a.mean(), a.std())

#### Step 3: Convert 3D array to 2D matrix

The next step is to convert our 3D array into a 2D matrix, so that we can perform matrix operations. We do this by combining all the spatial dimensions together.

First, let's check the shape of the data.

In [ ]:
a.shape

We have a 3-D array (time, latitude, longitude). We need to convert it to 2-D.  So, we combine the two spatial dimensions (latitude and longitude) into one by reshaping the arrays. Think of this as compressing or squeezing in the multidimensional data (time, lat, lon) into two dimensions (time, latlon).


In [ ]:

# Reshape the array to have one time dimension, one space dimension

Nt, Ny, Nx = a.sizes["time"], a.sizes["lat"], a.sizes["lon"]
A = a.values.reshape(Nt, Ny * Nx)

Check the shape of the combined data (`A.shape`).

In [ ]:
A.shape

#### Step 4: Calculate covariance matrix
Now, we can calculate the [covariance matrix](https://numpy.org/doc/2.3/reference/generated/numpy.cov.html). Since our data is standardized, we will actually be computing the correlation matrix.

Similar to the correlation matrix in Exercise 5, we are calculating for how similar (or different) are the SST observations in pairs of time and combined space. We won't be plotting the heatmap this time since this will be a correlation matrix for 2,821 pairs of data.

In [ ]:
C = np.cov(A, rowvar=False)
print(C.shape)

#### Step 5: Perform eigenanalysis of covariance matrix

Eigenanalysis finds the principal directions of *variability* in the covariance matrix - the axes along which the data vary the most.

In PCA, these directions are called the [*eigenvectors* (the spatial patterns or EOFs)](https://math.libretexts.org/Bookshelves/Linear_Algebra/A_First_Course_in_Linear_Algebra_(Kuttler)/07%3A_Spectral_Theory/7.01%3A_Eigenvalues_and_Eigenvectors_of_a_Matrix). Their corresponding *eigenvalues* tell us how much variance each mode explains.

In other words, Eigenvectors show the main directions of variability (the patterns), and eigenvalues tell how important each one is.

We can calculate these using [`np.linalg.eig()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html#scipy.linalg.eig), where the expected outputs are the Eigenvalues and the Eigenvectors.

In [ ]:
eigvals, eigvecs = np.linalg.eig(C)

The output of the eigenanalysis is a set of spatial patterns (EOFs). For the code above,

- `eigvals` → a list of numbers (one per mode) that tell us how much variance each eigenvector explains.
    - Larger eigenvalues → more important patterns.

- `eigvecs` → a set of vectors that show the directions or patterns of variability (in PCA, these become the EOFs).
    - Each column of `eigvecs` corresponds to one spatial pattern (one EOF).
    - Together with the time series they produce (the PCs), they describe how the data varies in space and time.

In short: `eigvals` tell us how much, and `eigvecs` tell us where and how the data vary.

We can try printing the `eigvals` array to show how important each principal component (PC) is. The magnitude of each element shows the *relative importance* of each PC (i.e. the first element corresponds to PC 1 (the dominant mode), the second to PC 2, and so on.).

In [ ]:
eigvals

The companion array `eigvecs` contains the spatial patterns (directions) associated with each PC.

Each **column** of `eigvecs` represents one eigenvector — the **pattern of weights** that defines how different grid points vary together to form that PC. 

Check the shape of `eigvecs` below.

In [ ]:
eigvecs.shape

The eigenvectors (`eigvecs`) are returned by NumPy as columns of a 2-D array, where each column corresponds to one principal component (PC), and each row corresponds to one grid point (a specific latitude–longitude location) in our dataset.

When we prepared our data for PCA, we *flattened the spatial dimensions* (lat × lon) into a single “space” dimension —
so each grid cell’s SST time series became *one row* in our covariance matrix.

In the next step, we’ll reshape these eigenvectors back into latitude–longitude maps to visualize the EOF spatial patterns.

#### Step 6: Extract and standardize the first two EOFs and PCs

The eigenvectors from the covariance matrix can also be called Empirical Orthogonal Functions (EOFs) in climate science.

Each EOF represents a spatial pattern that explains a particular fraction of variance in the dataset —
in our case, patterns of SST variability across the tropical Pacific.
- EOF 1 corresponds to the spatial pattern of the dominant mode (usually the seasonal cycle).
- EOF 2 often represents the next major mode (for SST, this is the ENSO pattern).

The term “Empirical” reflects that these patterns are derived from data, not theoretical equations — they are the data’s own preferred modes of variability.


Each EOF is a spatial pattern (the “where” of variability), and each PC is its time series (the “when”).
EOF1 and EOF2 are ordered by how much variance they explain — EOF1 explains the most, EOF2 the next.

To visualize an eigenvector (now an EOF) as a map again, we need to reshape it back from this flattened 1-D form into its original 2-D grid shape (Ny × Nx):

For example, in the code below
```python
EOF1 = np.real(eigvecs[:, 0]).reshape(Ny, Nx)
```
- `eigvecs[:, 0]` → takes the first eigenvector (corresponding to EOF 1)

- `.reshape(Ny, Nx)` → restores its 2-D structure so we can plot it on latitude–longitude coordinates




In [ ]:
# --- Extract first two EOF spatial patterns (eigenvectors reshaped to lat-lon grid) ---
EOF1 = np.real(eigvecs[:, 0]).reshape(Ny, Nx)
EOF2 = np.real(eigvecs[:, 1]).reshape(Ny, Nx)

After extracting the EOFs (the spatial patterns), we can find the corresponding Principal Components (PCs), the time series that tell us how strongly each pattern is expressed at each time step.

Mathematically, this is done using the dot product:

```python
PC1 = np.dot(A, np.real(eigvecs[:, 0]))
```

The dot product multiplies each grid point’s SST anomaly by its weight from the EOF pattern, then sums over all grid points. In other words, it projects the full SST field at each time step onto the EOF — telling us *“how much of that spatial pattern is present in the data at that moment."*

- `A` → SST anomaly data flattened into (time × space) form
- `eigvecs[:, 0]` → EOF 1 (spatial weights for each grid point)
- `np.dot(A, eigvecs[:, 0])` → multiplies each SST field by its weights → gives one value per time step (the PC time series)

Thus, the dot product converts spatial information (EOF) into temporal behavior (PC).

In [ ]:
# --- Compute corresponding principal component (PC) time series ---
PC1 = np.dot(A, np.real(eigvecs[:, 0]))
PC2 = np.dot(A, np.real(eigvecs[:, 1]))



After computing each Principal Component (PC) time series, we standardize them so they have a mean of 0 and a standard deviation of 1.

This step removes any offset or scale differences between components, making them directly comparable in amplitude and variability.

In [ ]:
# --- Standardize the PCs to zero mean and unit variance ---
PC1 = (PC1 - PC1.mean()) / PC1.std()
PC2 = (PC2 - PC2.mean()) / PC2.std()

#### Step 7. Plot EOF1 and PC1 (Seasonal Cycle mode)
- EOF 1 shows north–south temperature contrast: warm Southern Hemisphere vs cool Northern Hemisphere.
- PC 1 oscillates annually → this is the seasonal cycle emerging purely from statistics.
- PCA has identified the largest repeating pattern in the data — the annual temperature swing.

In [ ]:
plt.figure(figsize=(14,5))

# EOF 1 spatial pattern
plt.subplot(2,1,1)
plt.pcolormesh(a.lon, a.lat, EOF1, cmap="RdBu_r", vmin=-0.05, vmax=0.05)
plt.colorbar(label="Amplitude")
plt.title("EOF 1: Spatial Pattern (Seasonal Cycle)")

# PC 1 time series
plt.subplot(2,1,2)
plt.plot(PC1, color="r")
plt.title("PC 1: Time Series of Seasonal Cycle Mode")
plt.xlabel("Time (months)")
plt.ylabel("Standardized Value")
plt.tight_layout()
plt.show()


#### Step 8. Plot EOF 2 and PC 2 (ENSO / El Niño–La Niña Mode)
- EOF 2 shows the El Niño pattern —  warm anomalies in the central/eastern Pacific, cool anomalies in the west.
- PC 2 behaves similarly to the Niño 3.4 index (ENSO time series).
- Positive PC 2 → El Niño; Negative PC 2 → La Niña.
- PCA “discovered” ENSO without being told about it — that’s the power of statistical decomposition.

In [ ]:
plt.figure(figsize=(14,5))

# EOF 2 spatial pattern
plt.subplot(2,1,1)
plt.pcolormesh(a.lon, a.lat, EOF2, cmap="RdBu_r", vmin=-0.05, vmax=0.05)
plt.colorbar(label="Amplitude")
plt.title("EOF 2: Spatial Pattern (ENSO Mode)")

# PC 2 time series
plt.subplot(2,1,2)
plt.plot(PC2, color="b")
plt.title("PC 2: Time Series of ENSO Mode")
plt.xlabel("Time (months)")
plt.ylabel("Standardized Value")
plt.tight_layout()
plt.show()


#### Step 9 - Interpreting and Adjusting the Sign of EOFs

When performing Principal Component Analysis (PCA) or Empirical Orthogonal Function (EOF) analysis,
the resulting spatial patterns (EOFs) and corresponding time series (PCs) are unique only up to a sign.

Mathematically, both of these are valid solutions: (EOF,PC) and (−EOF,−PC)

This means that sometimes your EOF 2 map might appear “flipped” — for example, mostly blue when you expected red — because PCA does not assign physical meaning to “positive” or “negative”.

What matters are:
- the pattern of variability (warm vs. cool regions), and
- how that pattern co-varies with its time series.

To make the EOF interpretation consistent with known climate indices (e.g., Niño 3.4),
we usually flip the sign so that positive PC values correspond to El Niño (warm eastern Pacific)
and negative PC values correspond to La Niña (cool eastern Pacific).

Because PCA’s sign is arbitrary, we compare PC2 with the Niño 3.4 index and flip the sign if they’re anticorrelated (i.e. if they're inverted).
Here we used the overlapping 1950–2020 period, dropped missing values, and ensured both arrays had the same length before computing the correlation.

In [ ]:
plt.figure(figsize=(14,5))

# EOF 2 spatial pattern
plt.subplot(2,1,1)
# Take note of the negative sign before EOF2
plt.pcolormesh(a.lon, a.lat, -EOF2, cmap="RdBu_r", vmin=-0.05, vmax=0.05)
plt.colorbar(label="Amplitude")
plt.title("EOF 2: Spatial Pattern (ENSO Mode)")

# PC 2 time series
plt.subplot(2,1,2)
# Take note of the negative sign before PC2
plt.plot(-PC2, color="b")
plt.title("PC 2: Time Series of ENSO Mode")
plt.xlabel("Time (months)")
plt.ylabel("Standardized Value")
plt.tight_layout()
plt.show()


#### Step 10 - Comparing the PCA-Derived ENSO Mode with the Niño 3.4 Index

In the final step, we compare the second principal component (PC2) from our EOF analysis with the Niño 3.4 Index that we computed earlier from the SST anomalies.

Both describe the same underlying phenomenon — the El Niño–Southern Oscillation (ENSO) — but they are derived in very different ways:

- Niño 3.4 Index: a regional average of SST anomalies within 5° S–5° N and 170° W–120° W It’s a physically defined measure based on a fixed geographic box.
- PC 2: a statistical mode obtained through Principal Component Analysis of the entire tropical Pacific SST field. PCA identifies patterns that explain the largest fractions of variance, without any prior knowledge of ENSO.

By aligning their time periods and flipping the EOF 2/PC2 signs if needed, we find that these two time series are strongly correlated (typically r ≈ 0.7–0.9).

This strong agreement shows that PCA objectively “rediscovers” ENSO as the second dominant mode of tropical Pacific variability, right after the seasonal cycle.

In [ ]:
# Flip the sign of PC2
PC2_fl = -PC2

# Select overlapping period
nino34_sub = nino34_std.sel(time=slice("1950", "2020"))

# Drop NaNs (from running mean)
nino34_sub = nino34_sub.dropna("time")

# Match lengths
minlen = min(len(PC2_fl), len(nino34_sub))
pc2_aligned = PC2_fl[:minlen]
nino34_aligned = nino34_sub.values[:minlen]
time_aligned = nino34_sub.time[:minlen]

corr = np.corrcoef(pc2_aligned, nino34_aligned)[0, 1]
print(f"Correlation between Niño 3.4 index and PC 2 = {corr:.2f}")

import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))
plt.plot(time_aligned, nino34_aligned, color="r", label="Niño 3.4 Index")
plt.plot(time_aligned, pc2_aligned, color="b", label="PC 2 (EOF ENSO Mode)")
plt.axhline(0, color="gray", lw=0.8)
plt.legend()
plt.title(f"Niño 3.4 vs PCA-Derived PC 2 (1950–2020)  —  r = {corr:.2f}")
plt.ylabel("Standardized Anomaly (σ)")
plt.xlabel("Time")
plt.grid(alpha=0.3)
plt.show()


---

## Wrapping Up

In this lesson, we explored how Principal Component Analysis (PCA) (also known in meteorology as Empirical Orthogonal Function (EOF) analysis) can extract dominant patterns of variability from large, gridded climate datasets.

We began by:
1. Computing SST anomalies relative to a climatology.

2. Deriving the Niño 3.4 Index from a fixed tropical Pacific region.

3. Applying PCA to the full tropical Pacific SST field (1950 – 2020).

Our results showed that:

1. EOF 1 / PC 1 captures the annual (seasonal) cycle, which dominates total variance.
2. EOF 2 / PC 2 represents the El Niño–Southern Oscillation (ENSO) pattern—warm anomalies in the central and eastern Pacific (El Niño) and cool anomalies (La Niña).

After aligning signs, PC 2 strongly correlates with the Niño 3.4 Index (r ≈ 0.7–0.9), demonstrating that PCA statistically rediscovers ENSO without prior geographic constraints.

Key insight:
1. PCA identifies where and how the system varies together.
2. It transforms high-dimensional SST data into a few meaningful spatial modes and their corresponding temporal behaviors.


---

### PCA and FFT

Both PCA and FFT are decomposition techniques—but they answer different scientific questions.

| Feature             | **PCA / EOF Analysis**                                     | **FFT / Spectral Analysis**                            |
| ------------------- | ---------------------------------------------------------- | ------------------------------------------------------ |
| **Goal**            | Find dominant *spatial* or *covariance* patterns           | Find dominant *temporal* frequencies                   |
| **Basis functions** | Data-derived empirical patterns (EOFs)                     | Fixed sine + cosine waves                              |
| **Input**           | Multivariate fields (lat × lon × time)                     | Single or multivariate time series                     |
| **Output**          | EOFs → spatial modes; PCs → time series                    | Amplitude & phase spectra vs. frequency                |
| **Assumptions**     | No assumption of periodicity; patterns emerge from data    | Assumes stationarity and periodicity                   |
| **Best for**        | Discovering climate modes (ENSO, NAO, monsoon variability) | Detecting cycles (annual, diurnal, 3-7 yr ENSO period) |

In essence:

- PCA decomposes variance in space + time.
- FFT decomposes variance in time + frequency.

Together, they complement each other:
PCA tells us what spatial patterns vary together, while FFT tells us how often those variations occur.